# SAP Repro

Trying to reproduce SAP issues.

## Constants & Functions

In [13]:
HOST = '127.0.0.1'
PORT = 2223
USER = 'vagrant'
PASS = 'vagrant'

In [14]:
from ncclient import manager
from lxml import etree

def pretty_print(retval):
        print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    device_params={'name':'iosxe'},
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

In [15]:
vrf = '''
<nc:config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <vrf>
      <definition>
        <name>TEST</name>
        <rd>100:100</rd>
        <address-family>
          <ipv4/>
        </address-family>
      </definition>
    </vrf>
  </native>
</nc:config>
'''

vrf_del = '''
<nc:config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <vrf>
      <definition nc:operation="delete">
        <name>TEST</name>
      </definition>
    </vrf>
  </native>
</nc:config>
'''

def merge_vrf(conn):
    return conn.edit_config(vrf, format='xml', target='candidate')

def delete_vrf(conn):
    return conn.edit_config(vrf_del, format='xml', target='candidate')

In [17]:
#
# loop creating & deleting the vrf TEST
#
m.lock(target='running')
m.lock(target='candidate')
for _ in range(0, 1):
    print(merge_vrf(m).ok)
    # print(delete_vrf(m).ok)
m.commit()
m.unlock(target='candidate')
m.save_config()
m.unlock(target='running')


True


<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:b881b21c-ccab-4f64-8093-a3e3027e2353" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

In [32]:
vrf_route = '''
<nc:config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <ip>
      <route>
        <vrf>
          <name>TEST</name>
          <ip-route-interface-forwarding-list>
            <prefix>192.168.1.0</prefix>
            <mask>255.255.255.0</mask>
            <fwd-list>
              <fwd>10.10.10.1</fwd>
            </fwd-list>
          </ip-route-interface-forwarding-list>
        </vrf>
      </route>
    </ip>
  </native>
</nc:config>
'''

vrf_route_del = '''
<nc:config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <ip>
      <route>
        <vrf nc:operation="delete">
          <name>TEST</name>
        </vrf>
      </route>
    </ip>
  </native>
</nc:config>
'''

def merge_vrf_route(conn):
    return conn.edit_config(vrf_route, format='xml', target='running')

def delete_vrf_route(conn):
    return conn.edit_config(vrf_route_del, format='xml', target='running')

In [38]:
merge_vrf(m)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:9d0e4bf2-e72b-4ab4-891e-d826424d1a22" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

In [40]:
delete_vrf(m)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:f541dd2f-8ded-4e99-9a97-b7adedddb012" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

In [39]:
merge_vrf_route(m)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:ea5b3ec9-62db-4375-89eb-b1bcf61c8fb4" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

In [35]:
delete_vrf_route(m)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:df9c7888-6959-4edf-9e89-6d971233fa7e" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>